In [1]:
import requests
import urllib
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [2]:
#on crée une fonction avec les instructions du scrap

def scrap(url) : 
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')

    divs_prix = soup.find_all(class_="ergov3-priceannonce")
    
    
    for i in divs_prix:
        liste_prix.append(list(i.stripped_strings)[0])
    
    divs_desc = soup.find_all(class_="ergov3-h3")
    
    for i in divs_desc:
        x = list(i.stripped_strings)[0]
        x= re.sub(r"\t", "" ,  x)
        x= re.sub(r"\r", " " ,  x)
        x= re.sub(r"\n", "" ,  x)    
        liste_desc.append(x)

    villes = soup.find_all("cite")
    

    for x in villes:
        x= re.sub(r"\t", "" ,  x.text)
        x= re.sub(r"\r", " " ,  x)
        x= re.sub(r"\n", "" ,  x) 
        liste_villes.append(x)
    
    divs_liens = soup.find_all("a", class_="voirann")
    liste_liens_temp = []
    for i in divs_liens :
        liste_liens_temp.append(i.get('href'))
    liste_liens_temp = liste_liens_temp[7:]
    liste_liens.extend(liste_liens_temp)



In [3]:
#on initialiste les listes

liste_liens = []
liste_liens_temp =[]
liste_prix = []
liste_villes = []
liste_desc = []

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
          'referer':'https://www.google.com/'}



#1ere itération

url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?tt=1&at=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50"
scrap(url)
time.sleep(0.6)

#100 autres pages

for i in range (2,100) :
    url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?tt=1&at=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50&p="+str(i)
    scrap(url)
    time.sleep(0.6)


df = pd.DataFrame(columns =["intitule","ville","prix",'lien'])

df["intitule"]= liste_desc
df["ville"]= liste_villes
df["prix"]= liste_prix
df['lien'] = liste_liens

# spliter les intitulées en type, np p, surf




In [4]:
#on clean les lignes de pubs Vente

df = df[~df['intitule'].astype(str).str.startswith('V')]

#on sépare la colonne intitulé en deux

df[['type', 'taille']] = df['intitule'].str.split(',', expand=True)

df[['type de bien','nb de pièces','piece', 'none']] = df['type'].str.split(' ', expand=True)
df = df.drop(axis=1, columns=["intitule","type","piece","none"])

df


,ville,prix,lien,taille,type de bien,nb de pièces
0,Vaires-sur-Marne (77360),269 000 €,/immobilier/vente/appartement/maison-brou-sur-...,67 m²,Appartement,4
1,Palaiseau (91120),420 000 €,/immobilier/vente/maison-a-palaiseau/particuli...,91 m²,Maison,4
2,Conflans-Sainte-Honorine (78700),225 000 €,/immobilier/vente/appartement/appartement-conf...,72 m²,Appartement,4
3,Villeneuve-Saint-Georges (94190),210 000 €,/immobilier/vente/appartement/vente-appartemen...,58 m²,Appartement,3
4,Paris 15,599 000 €,/immobilier/vente/appartement/75015-paris-vaug...,56 m²,Appartement,3
...,...,...,...,...,...,...
2668,Champagne-sur-Oise (95660),263 000 €,/immobilier/vente/appartement/appartement-f3-c...,65 m²,Appartement,3
2669,Argenteuil (95100),170 000 €,/immobilier/vente/appartement/f2-lumineux/1262...,51 m²,Appartement,2
2670,Parmain (95620),365 000 €,/immobilier/vente/maison/ronquerolles/12627160...,93 m²,Maison,4
2671,Parmain (95620),455 000 €,/immobilier/vente/maison/maison-bruyeres-sur-o...,183 m²,Maison,7


In [7]:
#cleaning de la taille

df["carrez"]= df["taille"].str.contains("carrez")
df["taille"] = df["taille"].str.removesuffix("carrez")
df["taille"] = df["taille"].str.removesuffix("m² ")
df = df[df["taille"].notnull()]
df["taille"] = df["taille"].astype(int)

#cleaning du prix

df['prix'] = df['prix'].str.replace("€","").str.replace(" ","")
df['prix'] = df['prix'].astype(int)

#ajout de la racine du lien

df['lien'] = df['lien'].apply(lambda x : "https://www.paruvendu.fr"+x)

C:\Users\Syl\AppData\Local\Temp/ipykernel_19440/2778295025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["carrez"]= df["taille"].str.contains("carrez")
C:\Users\Syl\AppData\Local\Temp/ipykernel_19440/2778295025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["taille"] = df["taille"].str.removesuffix("carrez")
C:\Users\Syl\AppData\Local\Temp/ipykernel_19440/2778295025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

AttributeError: Can only use .str accessor with string values!

In [17]:
# on sauvegarde 

df.to_csv("scrap.csv")